# Rebuttal Experiments

## GSM8k Experiments

We are running MARL-Focal on GSM8k dataset using the same model pool with LLM-TOPLA:

```
['Llama-2-13b-hf', 'Llama-2-70b-hf', 'Llama-2-7b-hf', 'Mistral-7B-Instruct-v0.2', 'Mixtral-8x7B-v0.1', 'gemma-2b', 'gemma-7b', 'phi-2']
```

Here we are running experiments 5 times and report the mean accurcy in test set.

In [1]:
import sys
import argparse

# Running with the same model pool as LLM-TOPLA.
parser = argparse.ArgumentParser()

parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--model_names", type=str, default="all")
parser.add_argument("--dataset_type", type=str, default="lang", choices=["lang", "vision"])
parser.add_argument("--task_name", type=str, default="gsm8k", choices=["gsm8k", "mmlu_hf", "bbh", "gpqa", "musr"])

# select agent arguments
parser.add_argument("--alpha", type=float, default=1.0, help="size penalty constant")
parser.add_argument("--select_agent_epoch", type=int, default=10)
parser.add_argument("--select_agent_gamma", type=float, default=0.99)
parser.add_argument("--select_agent_clip_epsilon", type=float, default=0.2)
parser.add_argument("--select_agent_lr", type=float, default=0.001)
parser.add_argument("--focal_div_weight", type=float, default=0.5)
parser.add_argument("--plurality_voting_weight", type=float, default=0.5)
parser.add_argument("--fleiss_kappa_weight", type=float, default=0)
parser.add_argument("--q_stats_weight", type=float, default=0)
parser.add_argument("--corr_coef_weight", type=float, default=0)
parser.add_argument("--binary_disagreement_weight", type=float, default=0)
parser.add_argument("--kappa_stats_weight", type=float, default=0)
parser.add_argument("--binary_entropy_weight", type=float, default=0)

# ensemble agent arguments
parser.add_argument("--ensemble_agent_epoch", type=int, default=10)
parser.add_argument("--ens_agent_gamma", type=float, default=0.5)
parser.add_argument("--ens_agent_clip_epsilon", type=float, default=0.2)
parser.add_argument("--ens_agent_lr", type=float, default=0.001)

# common argumentss for both agents
parser.add_argument("--window_size", type=int, default=500)
parser.add_argument("--max_tolerance", type=int, default=150)
parser.add_argument("--update_freq", type=int, default=100)


# Simulate command-line arguments
if 'ipykernel' in sys.modules:
    sys.argv = ['notebook.py']  # Replace with desired arguments
else:
    sys.argv = ['script.py']

# Parse the arguments
args = parser.parse_args()

In [2]:
from run import main

test_scores = []
for i in range(5):
    test_score = main(args)
    test_scores.append(test_score)

/home/hice1/stekin6/.conda/envs/llamas/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Llama-2-13b-chat-hf', 'Llama-2-70b-chat-hf', 'Llama-2-7b-chat-hf', 'Mistral-7B-Instruct-v0.2', 'Mixtral-8x7B-Instruct-v0.1', 'gemma-2b', 'gemma-7b', 'phi-2']
Truncating the space size
Truncating the space size
Dataset-0: gsm8k


100%|██████████| 2748/2748 [02:40<00:00, 17.15it/s]


Episode: 0, Total Acc: 72.48%, Action Counts: [2095. 1966. 1165.  489. 1297. 1550. 2045. 1336.], Avg Inf Time: 0.05776s


100%|██████████| 2748/2748 [02:27<00:00, 18.61it/s]


...

Episode: 8, Total Acc: 0.40%, Action Counts: [ 227. 2048. 2270.  701. 1145.  774. 2545. 2390.], Avg Inf Time: 0.04082s


100%|██████████| 2748/2748 [01:48<00:00, 25.22it/s]


Episode: 9, Total Acc: 38.70%, Action Counts: [ 244. 2077. 2308.  735. 1075.  766. 2548. 2398.], Avg Inf Time: 0.03926s
Starting testing


100%|██████████| 1240/1240 [01:11<00:00, 17.27it/s]


Episode: 0, Total Acc: 76.35%, Action Counts: [ 127.  881. 1031.  290.  520.  361. 1169. 1074.], Avg Inf Time: 0.05762s


Test Acc: 76.35%
Data is saved...


In [2]:
print(test_scores)

[76.43, 77.32, 74.09, 73.45, 76.35]


In [6]:
import numpy as np
print(np.mean(test_scores))

75.728


## MMLU Experiments with Additional 4 Models

In addition to the model pool in the paper,
```
['Llama-2-13b-hf', 'Llama-2-70b-hf', 'Llama-2-7b-hf', 'Mistral-7B-Instruct-v0.2', 'Mixtral-8x7B-v0.1', 'gemma-2b', 'gemma-7b', 'phi-2']
```

 we adding 4 strong models, 'Meta-Llama-3-70B', 'Mixtral-8x22B-v0.1', 'Qwen-72B', and 'deepseek-llm-67b' to create a diverse pool containing 12 models.

 ```
['Llama-2-13b-hf', 'Llama-2-70b-hf', 'Llama-2-7b-hf', 'Meta-Llama-3-70B', 'Mistral-7B-Instruct-v0.2', 'Mixtral-8x22B-v0.1', 'Mixtral-8x7B-v0.1', 'Qwen-72B', 'deepseek-llm-67b', 'gemma-2b', 'gemma-7b', 'phi-2']
```

Below we show the test accuracy scores of base models predictions:


In [1]:
import numpy as np
from data_generator.data_loader import DataCreator

datacreator = DataCreator(dataset_name="mmlu_hf")

for train_data, test_data, num_models, ds_name in datacreator.load():
    print(train_data.shape, test_data.shape)
    space_size = (train_data.shape[1] - 1) // num_models

    novel_data = test_data[:, :-1]
    novel_label = test_data[:, -1]
    acc = []
    for i, novel_arr in enumerate(np.split(novel_data, num_models, axis=1)):
        pred = novel_arr.argmax(axis=1)
        model_acc = np.mean(novel_label == pred)
        acc.append(model_acc)
    acc = np.array(acc)
    print()
    for i in range(len(acc)):
        print(f"{datacreator.model_names[i]}: {acc[i] * 100:.2f}")

/home/selim/miniconda3/envs/mistral/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Llama-2-13b-hf', 'Llama-2-70b-hf', 'Llama-2-7b-hf', 'Meta-Llama-3-70B', 'Mistral-7B-Instruct-v0.2', 'Mixtral-8x22B-v0.1', 'Mixtral-8x7B-v0.1', 'Qwen-72B', 'deepseek-llm-67b', 'gemma-2b', 'gemma-7b', 'phi-2']
(11233, 49) (2809, 49)

Llama-2-13b-hf: 53.40
Llama-2-70b-hf: 68.53
Llama-2-7b-hf: 41.79
Meta-Llama-3-70B: 77.29
Mistral-7B-Instruct-v0.2: 59.67
Mixtral-8x22B-v0.1: 76.36
Mixtral-8x7B-v0.1: 70.42
Qwen-72B: 75.01
deepseek-llm-67b: 71.24
gemma-2b: 40.26
gemma-7b: 63.87
phi-2: 55.82


### MMLU MARL-Focal with the new model pool

In [ ]:
import sys
import argparse

# Running with the same model pool as LLM-TOPLA.
parser = argparse.ArgumentParser()

parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--model_names", type=str, default="all")
parser.add_argument("--dataset_type", type=str, default="lang", choices=["lang", "vision"])
parser.add_argument("--task_name", type=str, default="mmlu_hf", choices=["gsm8k", "mmlu_hf", "bbh", "gpqa", "musr"])

# select agent arguments
parser.add_argument("--alpha", type=float, default=1.0, help="size penalty constant")
parser.add_argument("--select_agent_epoch", type=int, default=15)
parser.add_argument("--select_agent_gamma", type=float, default=0.99)
parser.add_argument("--select_agent_clip_epsilon", type=float, default=0.2)
parser.add_argument("--select_agent_lr", type=float, default=0.001)
parser.add_argument("--focal_div_weight", type=float, default=0.5)
parser.add_argument("--plurality_voting_weight", type=float, default=0.5)
parser.add_argument("--fleiss_kappa_weight", type=float, default=0)
parser.add_argument("--q_stats_weight", type=float, default=0)
parser.add_argument("--corr_coef_weight", type=float, default=0)
parser.add_argument("--binary_disagreement_weight", type=float, default=0)
parser.add_argument("--kappa_stats_weight", type=float, default=0)
parser.add_argument("--binary_entropy_weight", type=float, default=0)

# ensemble agent arguments
parser.add_argument("--ensemble_agent_epoch", type=int, default=15)
parser.add_argument("--ens_agent_gamma", type=float, default=0.5)
parser.add_argument("--ens_agent_clip_epsilon", type=float, default=0.2)
parser.add_argument("--ens_agent_lr", type=float, default=0.001)

# common argumentss for both agents
parser.add_argument("--window_size", type=int, default=500)
parser.add_argument("--max_tolerance", type=int, default=150)
parser.add_argument("--update_freq", type=int, default=100)


# Simulate command-line arguments
if 'ipykernel' in sys.modules:
    sys.argv = ['notebook.py']  # Replace with desired arguments
else:
    sys.argv = ['script.py']

# Parse the arguments
args = parser.parse_args()